In [2]:
productsData = ""
with open("./data/productsData.txt","r") as productFile:
   productsData = productFile.read()
print(productsData)

*********
customer care number or human representative contact information:
email : contact@brainx.com 
our representative would contact you shortly
*********
generative ai model that we are using : gemini-pro
cost : 1$ per customer served
flexible with cost based on customer requirements
*********
no response :
check for stable internet connectivity
if internet is stable check for any warning message that appears on the screen
*********
warning message : 404 unstable internet connection
it could be caused because of interet issue
try turning internet off and then on again
*********
warning message : 300 unable to serve at the moment
it possibly mean that our servers are facing some critical issues at the moment 
we will try to get it fix as fast as possible
*********
warning message : 200 thanks for connecting
it means that the conversation was successfull and you can logout from your device
*********
commision on selling product : 
we offer 5% of the deal if person joins through your

In [20]:
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv
import requests

from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv
from langchain import OpenAI
import openai
from langchain_community.vectorstores import Milvus
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
from langchain.memory import PostgresChatMessageHistory

load_dotenv(override=True)

True

In [4]:
def get_docs(collection_name):
    # data = get_data(collection_name)
    text_splitter = CharacterTextSplitter(
        separator = "*********",
        chunk_size=400,
        chunk_overlap=0
    )
    dataDocument = [Document(page_content=productsData, metadata={"namespace": collection_name})]
    docs = text_splitter.split_documents(dataDocument)
    return docs



def create_vectorDB(collection_name):
    docs = get_docs(collection_name)
    initialize_embeddings()
    global vector_db
    vector_db = Milvus.from_documents(
        docs,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
            "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
            "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
            # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
            "secure": True,
        },
    )


def initialize_embeddings():
    try :
        global embeddings
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name="sentence-transformers/all-MiniLM-l6-v2"
        )
    except Exception as e :
        print("probelm accured while creating embeddings \n{}".format(e))


In [5]:
# create_vectorDB("callbot")
initialize_embeddings()

In [12]:
text_splitter = CharacterTextSplitter(
    separator = "*********",
    chunk_size=400,
    chunk_overlap=0
)
dataDocument = [Document(page_content=productsData, metadata={"namespace": "simm"})]
docs = text_splitter.split_documents(dataDocument)

In [13]:
docs

[Document(page_content='customer care number or human representative contact information:\nemail : contact@brainx.com \nour representative would contact you shortly\n*********\ngenerative ai model that we are using : gemini-pro\ncost : 1$ per customer served\nflexible with cost based on customer requirements', metadata={'namespace': 'simm'}),
 Document(page_content='no response :\ncheck for stable internet connectivity\nif internet is stable check for any warning message that appears on the screen\n*********\nwarning message : 404 unstable internet connection\nit could be caused because of interet issue\ntry turning internet off and then on again', metadata={'namespace': 'simm'}),
 Document(page_content='warning message : 300 unable to serve at the moment\nit possibly mean that our servers are facing some critical issues at the moment \nwe will try to get it fix as fast as possible\n*********\nwarning message : 200 thanks for connecting\nit means that the conversation was successfull a

In [14]:
def load_vectorDB(collection_name):
    global vector_db
    vector_db = Milvus(
    embeddings,
    collection_name=collection_name,
    connection_args={
    "uri":os.getenv("ZILLIZ_CLOUD_URI") ,
    "user":os.getenv("ZILLIZ_CLOUD_USERNAME") ,
    "password":os.getenv("ZILLIZ_CLOUD_PASSWORD") ,
    # "token": ZILLIZ_CLOUD_API_KEY,  # API key, for serverless clusters which can be used as replacements for user and password
    "secure": True,
    },
)
    
load_vectorDB("callbot")


In [16]:
vector_db.similarity_search(query="can i contact human" , k=10)

[Document(page_content='customer care number or human representative contact information:\nemail : contact@brainx.com \nour representative would contact you shortly\n*********\ngenerative ai model that we are using : gemini-pro\ncost : 1$ per customer served\nflexible with cost based on customer requirements', metadata={'namespace': 'callbot'}),
 Document(page_content='I want to visit your office:\noffice location : 6A, south evenue, 12th street delhi.\nyou can also request online meeting at https://www.brainBotX.com/contact\n*********\nhow to switch ai model: \nopen your profile,\ngo to dashboard,\nunder Models sectoion select "let me choose model"\nchoose your preffered model', metadata={'namespace': 'callbot'}),
 Document(page_content='commision on selling product : \nwe offer 5% of the deal if person joins through your referral link\n*********\nhow to signup or register for the chat bot or call bot services :\nyou can go to our official website https://www.brainBotX.com/signup to r

In [150]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY") , temperature=0.6)


In [177]:

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI


model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
query = "how are you"
tags = model(
    [
        SystemMessage(content="you are a tag generator. you have to assign tags to user query which can be used to suggest products to the customer .do not type any other things only provide tags. tags could be any color , size , gender ,cloth fabric in [Cotton polyester Silk Blend Acrylic Wool Jean Polyester Faux Georgette Silk Jacquard] or design catagory in [Casual Sports Regular Birthday Formal Clubwear Partywear Funky Sweater casual Aesthetic Sport traditional]. if you do not find any relavant tags then return None."),
        HumanMessage(content=query),
    ]
)

In [178]:
print(tags.content)


None


In [162]:
simtag = vector_db.similarity_search(query=tags.content , k=10)

In [163]:
simtag

[Document(page_content='product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57%', metadata={'namespace': 'similarity'}),
 Document(page_content="product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20%", metadata={'namespace': 'similarity'}),
 Document(page_content='product name : leotude oversized cottonblend half sleeve back printed drop shoulder men t-shirt company brand : leotude item color black blue style look : funky gender : men cloth fabric : cotton blend discounted price : 289 discount : 74%', metadata={'namespace': 'similarity'}),
 Document(page_content='product na

In [164]:
simq = vector_db.similarity_search(query=query , k=10)

In [165]:
simq

[Document(page_content="product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20%", metadata={'namespace': 'similarity'}),
 Document(page_content='product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57%', metadata={'namespace': 'similarity'}),
 Document(page_content='product name : pipasa women woolen buttoned collar neck stylish party winter wear coat cardigan sweater company brand : pipasa item color brown grey pista red style look : sweater winter gender : women cloth fabric : wool discounted price : 899 discount : 55%', metadata={'namespace': 'similarity'}),
 Document(page

In [166]:
contextt = simtag[0].page_content + " --- " + simtag[1].page_content + " --- " + simtag[2].page_content + " --- " + simq[0].page_content + " --- " + simq[1].page_content + " --- " + simq[2].page_content

In [167]:
contextt

"product name : ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure company brand : ellipse item color green style look : winter gender : women cloth fabric : nylon discounted price : 2,160 discount : 57% --- product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men cloth fabric : polyester discounted price : 799 discount : 20% --- product name : leotude oversized cottonblend half sleeve back printed drop shoulder men t-shirt company brand : leotude item color black blue style look : funky gender : men cloth fabric : cotton blend discounted price : 289 discount : 74% --- product name : abbl fashionable full sleeves winter men's standard length polyester jacket for stylish comfortable huddy for adult boys company brand : abbl item color black grey style look : winter gender : men clot

In [168]:
llm.invoke("""you are a salse person. try to give human like response according to user query and given context.
    Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
    ------
    <ctx>
    {}
    </ctx>
    ------
    <hs> </hs>
    {}
    Answer:""".format(contextt,query ))

"I understand you're feeling cold. May I suggest taking a look at our warm and stylish winter jackets? We have a variety of options to choose from, so you're sure to find one that's perfect for you."

In [170]:
llm.invoke("""you are a salse person. try to give human like response according to user query and given context.
    Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
    ------
    <ctx>
    {}
    </ctx>
    ------
    <hs>
    bot : "I understand you're feeling cold. May I suggest taking a look at our warm and stylish winter jackets? We have a variety of options to choose from, so you're sure to find one that's perfect for you."
    user : "yes sure"
    bot : "Absolutely! Let me show you our collection of winter jackets. We have a wide range of styles and colors to choose from, so you're sure to find the perfect one for your needs."
    </hs>
    {}
    Answer:""".format(contextt,"show me the best" ))

"Of course! Our most popular winter jacket is the ellipse jackets for women full sleeves winter jackets with inner mobile pocket jackets with zip closure. It's made from high-quality nylon and features a stylish design. It's also water-resistant and windproof, so you'll stay warm and dry even in the coldest weather."

In [ ]:
{
    title: "Product 1",
    description: "Description of Product 1",
    imageURL: "https://example.com/product1.jpg"
},  

In [195]:
dff = pd.read_csv("data/finalDataset.csv")
# dff = dff.dropna()

In [196]:
dff.dropna()

,Unnamed: 0,Type,ProductBrand,Color,Size,LookBook,Gender,Fabric,ImageLink,Season,DiscountedPrice,Discount,ProductLink


In [197]:
dff.head()

,Unnamed: 0,Type,ProductBrand,Color,Size,LookBook,Gender,Fabric,ImageLink,Season,DiscountedPrice,Discount,ProductLink
0,0,nike men's relaxed fit t-shirt,nike,gold,S,casual,men,cotton,https://m.media-amazon.com/images/I/51UAabRZV2...,NaN,"2,489",47%,https://www.amazon.in/Nike-Relaxed-T-Shirt-DJ7...
1,1,puma men's regular fit t-shirt,puma,black white,S,sports,men,cotton,https://m.media-amazon.com/images/I/513rGsnZbo...,NaN,"1,439",20%,https://www.amazon.in/Puma-Printed-Regular-T-S...
2,2,nike women shirt,nike,black,M,sports,women,polyester,https://m.media-amazon.com/images/I/41h6g+dkZd...,NaN,809,55%,https://www.amazon.in/Nike-womens-Slim-Shirt-D...
3,3,puma men sweatshirt,puma,gray,XS,regular,men,cotton,https://m.media-amazon.com/images/I/61WRIRaAJp...,NaN,"1,499",50%,https://www.amazon.in/Puma-Cotton-Sweatshirt-8...
4,4,nike men casual shorts,nike,blue black,XS,casual,men,polyester,https://m.media-amazon.com/images/I/81NDwU5zk2...,NaN,"1,349",46%,https://www.amazon.in/Nike-Boxer-Shorts-DQ4813...


In [198]:
prod = []
for i in range(len(dff)) :
    prod.append({
        "title" : dff["Type"][i] , 
        "description" : dff["Fabric"][i] ,
        "imageURL" : dff['ImageLink'][i] ,
        "productURL" : dff['ProductLink'][i] ,
        "price" : dff['DiscountedPrice'][i] ,
        "discount" : dff['Discount'][i] ,
    })
    # print(prod)

# v2

In [ ]:
tag_dictionary = {
    "summer" : ["cotton" , "shorts" , "relax fit" , "loose fit" , "light color" , "white" , "tank tops", "summer" , "summer shorts" ],
    "party wear" : ["funky" , 'partywear' , "party wear" , "aesthetic" , "silk"],
    "winter" : ["sweater" , 'sweat shirt' , "jacket" , "thermal clothes"],
    "marriage" : ["traditional" , "saree" , "lehenga" , "kurta" , "dhoti" ],
}

In [212]:
qq = ''' 
available filters : {"price range": null , "color" : null, "design look" : null,"gender" : null  }
available options for color : "black" , "white" , "blue" , "grey" , "red"
available options for gender : "men" , "women"  
available options for price range : "1000 - 5000" , "5000 - 10000" , "10000 - 20000",  "20000 - 30000" 
available options for design look : "casual" , "sports" , "regular" , "funky"  , "formal" , "sport" , "traditional" , "summer" , "partywear"
user query : summer night party
'''
filter = model(
    [
        SystemMessage(content="you are a filter generator. you have ability to apply and modify provided filters based on user query. you are given filter object below. you can also have multiple values for particular filter, which is most relevant to the user query, but all the values should be from provided options value. do not provide any values to filter when there is no relevant information is given about that filter. do not type any other things only provide updated filters. provide entire object. modify object only when needed."),
        HumanMessage(content=qq),
    ]
)

print(filter.content)

{"price range": null , "color" : null, "design look" : "summer" ,"gender" : null  }


In [211]:
df["LookBook"].unique()

array(['casual', 'sports', 'regular', 'birthday regular',
       'formal clubwear partywear', 'formal', 'funky', 'sweater winter',
       'aesthetic', 'sport', 'traditional', 'partywear', 'summer ',
       'summer loose-fit', 'winter', 'summer shorts',
       'bussiness smartlook marriage', 'funky aesthetic'], dtype=object)

In [208]:
df

,Type,ProductBrand,Color,LookBook,Gender,Fabric,DiscountedPrice,Discount,str
0,nike men's relaxed fit t-shirt,nike,gold,casual,men,cotton,"2,489",47%,product name : nike men's relaxed fit t-shirt...
1,puma men's regular fit t-shirt,puma,black white,sports,men,cotton,"1,439",20%,product name : puma men's regular fit t-shirt...
2,nike women shirt,nike,black,sports,women,polyester,809,55%,product name : nike women shirt company brand...
3,puma men sweatshirt,puma,gray,regular,men,cotton,"1,499",50%,product name : puma men sweatshirt company br...
4,nike men casual shorts,nike,blue black,casual,men,polyester,"1,349",46%,product name : nike men casual shorts company...
5,adidas men's regular track pants,adidas,black white,birthday regular,men,cotton,"1,799",40%,product name : adidas men's regular track pan...
6,u.s. polo assn. men's regular fit t-shirt,u.s. polo,maroon aqua olive red navy,regular,men,cotton,"1,297",41%,product name : u.s. polo assn. men's regular ...
7,allen solly men's regular fit polo,allen solly,red black white blue navy cyan green yellow,regular,men,cotton polyester,699,36%,product name : allen solly men's regular fit ...
8,cvc shining shirt | formal shirt | clubwear shirt,cvc,black white red pink,formal clubwear partywear,men,silk,599,80%,product name : cvc shining shirt | formal shi...
9,arrow men's solid white full sleeve slim fit c...,arrow,white,formal,men,cotton,"1,139",43%,product name : arrow men's solid white full s...


In [226]:
#  get tags
qqq = "suggest me cloths for my interview tomorrow"
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
filter = model(
    [
        SystemMessage(content="you are a tag generator. you have to provide tags which can be used to suggest clothing products to the customer. for example - summer, - party wear, - partywear, - winter, -marriage. suggest tags based on user query. do not type any other things only provide tags."),
        HumanMessage(content=qqq),
    ]
)
tag_dictionary = {
"summer" : ["cotton" , "shorts" , "relax fit" , "loose fit" , "light color" , "white" , "tank tops", "summer" , "summer shorts" ],
"partywear" : ["funky" , 'partywear' , "party wear" , "aesthetic" , "silk"],
"winter" : ["sweater" , 'sweat shirt' , "jacket" , "thermal clothes"],
"marriage" : ["traditional" , "saree" , "lehenga" , "kurta" , "dhoti" ],
}
tags = " "
for i in filter.content.split("-"):
    tags = tags + i.strip() + " "
    if i.strip() == "summer" :
        tags = tags + i.strip() + " " + " ".join(tag_dictionary["summer"])
    if i.strip() == "partywear" :
        tags = tags + i.strip() + " " + " ".join(tag_dictionary["partywear"])
    if i.strip() == "winter" :
        tags = tags + i.strip() + " " + " ".join(tag_dictionary["winter"])
    if i.strip() == "marriage" :
        tags = tags + i.strip() + " " + " ".join(tag_dictionary["marriage"])

    

        
# print("filter" , filter.content)
print("tags" , tags)

tags   formal professional interview attire business casual dress pants button down shirt blazer suit tie 
